# Análisis comentarios TikTok

In [14]:
# Importamos las librerías

import nltk
import pandas as pd
from datetime import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from nltk.tokenize import RegexpTokenizer

In [15]:
df_coments_a = pd.read_csv("tiktok_coments_lomeli.csv", usecols=["tiktok-j2a19r-SpanText", "tiktok-1p6dp51-StrongText"])
df_coments_b = pd.read_csv("tiktok_coments_lomeli_2.csv", usecols=["tiktok-j2a19r-SpanText", "tiktok-1p6dp51-StrongText"])

In [16]:
# Renombramos la columnas

df_coments_a = df_coments_a.rename(columns={"tiktok-j2a19r-SpanText": "texto", "tiktok-1p6dp51-StrongText": "texto2"})
df_coments_b = df_coments_b.rename(columns={"tiktok-j2a19r-SpanText": "texto", "tiktok-1p6dp51-StrongText": "texto2"})


In [17]:
# Unimos los textos

df_coments_a.fillna('')
df_coments_a["texto"] = df_coments_a.texto + ' ' + df_coments_a.texto2
df_coments_a = df_coments_a.drop("texto2", axis=1)

df_coments_b.fillna('')
df_coments_b["texto"] = df_coments_b.texto + ' ' + df_coments_b.texto2
df_coments_b = df_coments_b.drop("texto2", axis=1)

df_coments = pd.concat([df_coments_a, df_coments_b])
df_coments = df_coments.dropna()
df_coments['texto'] = df_coments['texto'].str.lower()
tokenizer = RegexpTokenizer(r'\w+')
df_coments['tokens'] = df_coments['texto'].apply(tokenizer.tokenize)
df_coments.to_csv("coments_tiktok.csv")
df_coments


,texto,tokens
0,📊 #jalisco2024,[jalisco2024]
1,empoderemos a las mujeres en jalisco con un al...,"[empoderemos, a, las, mujeres, en, jalisco, co..."
2,el doc #carloslomeli,"[el, doc, carloslomeli]"
3,mi hermoso colima #colima,"[mi, hermoso, colima, colima]"
4,"no se haga, doc #morena","[no, se, haga, doc, morena]"
5,¿quienes crees que ganen la candidatura a gobe...,"[quienes, crees, que, ganen, la, candidatura, ..."
6,está tirado en la hamaca en #movimientociudadano,"[está, tirado, en, la, hamaca, en, movimientoc..."
8,no siempre se puede pues. #morenajalisco,"[no, siempre, se, puede, pues, morenajalisco]"
11,niega @dr. carlos lomelí,"[niega, dr, carlos, lomelí]"
14,"no sólo será enfrentar a morena y amlo, tambié...","[no, sólo, será, enfrentar, a, morena, y, amlo..."
